# **Projekt SF1930 - LCQ Finalist Prediction**

**Syfte:**
Vi förutspår vilka skateboardåkare som kvalificerar sig till finalen i SLS Super Crown Championship genom statistisk modellering.

**Huvudsteg:**
- Normalisera trick- och run-betyg.
- Skatta sannolikheten att landa trick och trickpoäng.
- Använd momentmetoden för att estimera nyckelparametrar.
- Simulera LCQ-resultat 5000 gånger för att förutse finalister.

---

# **Uppgift 2a: Skattning av $\theta$ (Landningssannolikhet)**

**Vad är $\theta$?**
- $\theta$ är sannolikheten att en skateboardåkare landar ett trick.
- Vi använder **momentmetoden** för att skatta $\theta$ baserat på historisk data.

**Vad gör vi här?**
- Läser in data och filtrerar ut LCQ-deltagare.
- Beräknar andelen landade trick för varje åkare.
- Sparar resultaten i en parameter-tabell.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# Läs in data
df = pd.read_parquet("df.parquet.gzip").set_index("id")

# Definiera LCQ-deltagare
ids = ["Majerus", "Oliveira", "Decenzo", "Santiago", "Papa", "Eaton", "Mota", "Shirai", "Jordan", "Hoefler", "Hoban", "Gustavo", "Ribeiro C", "O’neill", "Foy", "Midler"]
makes = ["make 1", "make 2", "make 3", "make 4"]

# Filtrera datasetet för LCQ-åkare
LQC = df.loc[ids, :]
LQC_make = df.loc[ids, makes]

# Beräkna momentmetodsskattning av \theta
grouped_LQC_make = LQC_make.groupby("id").sum().sum(axis=1)
tricks_count = LQC.index.value_counts() * 4
theta = (grouped_LQC_make / tricks_count).reset_index()
theta.columns = ["id", "theta"]

# Visa några av de beräknade theta-värdena
print("Exempel på skattade theta-värden:")
print(theta.head())

Exempel på skattade theta-värden:
        id   theta
0  Decenzo  0.4375
1    Eaton  0.6250
2      Foy  0.5000
3  Gustavo  0.4000
4    Hoban  0.4000


# **Uppgift 2b: Skattning av $\alpha$ och $\beta$ (Trickprestanda)**

**Vad är $\alpha$ och $\beta$?**
- $\alpha$ och $\beta$ styr betafördelningen för trickpoäng.
- Högre $\alpha$ betyder oftare höga poäng, medan högre $\beta$ betyder oftare låga poäng.

**Vad gör vi här?**
- Beräknar första ($m_1$) och andra ($m_2$) momentet.
- Använder momentmetoden för att skatta $\alpha$ och $\beta$.
- Justerar parametrarna för stabilitet.

In [2]:
tricks = ["trick 1", "trick 2", "trick 3", "trick 4"]
LQC_tricks = LQC.loc[:, tricks]

sum_LQC_tricks = LQC_tricks.groupby("id").sum().sum(axis=1)
LQC_tricks_squared = LQC_tricks.apply(lambda x: x**2)
sum_LQC_tricks_squared = LQC_tricks_squared.groupby("id").sum().sum(axis=1)

m1 = sum_LQC_tricks / grouped_LQC_make
m2 = sum_LQC_tricks_squared / grouped_LQC_make

# Skatta alfa och beta
alfa_x = (m1 * (m1 * (1 - m1) / (m2 - m1**2) - 1)).reset_index()
beta_x = (((m1 - m2) * (1 - m1)) / (m2 - m1**2)).reset_index()

alfa_x.columns = ["id", "alfa"]
beta_x.columns = ["id", "beta"]

# Justering för Santiago
alfa_x.loc[alfa_x["id"] == "Santiago", "alfa"] = 1
beta_x.loc[beta_x["id"] == "Santiago", "beta"] = 1

# Visa några av de skattade parametrarna
print("Exempel på skattade alfa och beta-värden:")
print(alfa_x.head())
print(beta_x.head())

# Spara alla parametrar
alla_param = pd.DataFrame({"id": alfa_x["id"], "theta": theta["theta"], "alpha": alfa_x["alfa"], "beta": beta_x["beta"]})
alla_param.round(2).to_csv('alla2.csv', index=False)

Exempel på skattade alfa och beta-värden:
        id        alfa
0  Decenzo   24.455929
1    Eaton   75.408256
2      Foy   51.589516
3  Gustavo   70.644593
4    Hoban  107.698921
        id       beta
0  Decenzo   5.110825
1    Eaton  20.045233
2      Foy   9.104032
3  Gustavo  17.523386
4    Hoban  15.034892


# **Uppgift 2c: Skattning av $\alpha_y$ och $\beta_y$ (Runprestationer)**

**Vad gör vi här?**
- Beräknar momentmetodsskattningar för betafördelningen av run-betyg.


In [3]:
runs = ["run 1", "run 2"]
LQC_runs = LQC.loc[:, runs]

sum_LQC_runs = LQC_runs.groupby("id").sum().sum(axis=1)
LQC_runs_squared = LQC_runs.apply(lambda x: x**2)
sum_LQC_runs_squared = LQC_runs_squared.groupby("id").sum().sum(axis=1)
nr_runs = LQC_runs.index.value_counts() * 2

m1_y = sum_LQC_runs / nr_runs
m2_y = sum_LQC_runs_squared / nr_runs

alfa_y = (m1_y * (m1_y * (1 - m1_y) / (m2_y - m1_y**2) - 1)).reset_index()
beta_y = (((m1_y - m2_y) * (1 - m1_y)) / (m2_y - m1_y**2)).reset_index()

alfa_y.columns = ["id", "alfa"]
beta_y.columns = ["id", "beta"]

# Visa några av de skattade runparametrarna
print("Exempel på skattade runparametrar:")
print(alfa_y.head())
print(beta_y.head())

# Spara parametrar
alla_param_y = pd.DataFrame({"id": alfa_x["id"], "alpha": alfa_y["alfa"], "beta": beta_y["beta"]})
alla_param_y.round(2).to_csv('allay2.csv', index=False)

Exempel på skattade runparametrar:
        id        alfa
0  Decenzo    4.164506
1    Eaton  103.573288
2      Foy    3.506609
3  Gustavo    1.228081
4    Hoban    3.632207
        id       beta
0  Decenzo   2.834664
1    Eaton  36.865068
2      Foy   4.088934
3  Gustavo   0.856946
4    Hoban   2.096826


# **Uppgift 2d: Simulering av LCQ-finaler**

**Vad gör vi här?**
- Simulerar **5000 LCQ-tävlingar**.
- Beräknar skateboardåkarnas totala poäng.
- Identifierar de åkare som oftast kvalificerar sig till finalen.

In [4]:
from collections import Counter

# Skapa DataFrame för att lagra totalpoäng
total_grades = pd.DataFrame({"id": ids, "total_grade": None})
nested_finalists = []

# Antal simuleringar
num_simulations = 5000

for _ in range(num_simulations):
    for id in ids:
        # Simulera trick och runs
        V = np.random.binomial(1, theta.loc[theta["id"] == id, "theta"], 4)
        Z = np.random.beta(alfa_x.loc[alfa_x["id"] == id, "alfa"], beta_x.loc[beta_x["id"] == id, "beta"], 4)
        tricks = V * Z
        runs = np.random.beta(alfa_y.loc[alfa_y["id"] == id, "alfa"], beta_y.loc[beta_y["id"] == id, "beta"], 2)

        # Beräkna totalpoäng
        total_grade = tricks.max() + np.delete(tricks, np.argmax(tricks)).max() + runs.max()
        total_grades.loc[total_grades["id"] == id, "total_grade"] = total_grade

    # Sortera poängen och hämta finalisterna
    total_grades = total_grades.sort_values("total_grade", ascending=False)
    nested_finalists.append(total_grades.iloc[:4, 0].tolist())

# Räkna hur ofta varje skateboardåkare kvalificerar sig till finalen
skater_count = Counter([finalist for sublist in nested_finalists for finalist in sublist])
most_common_finalists = skater_count.most_common(4)

# Visa de mest frekventa finalisterna
print("Mest frekventa finalister:")
for skater, count in most_common_finalists:
    print(f"{skater}: {count} gånger")

Mest frekventa finalister:
Jordan: 2483 gånger
Eaton: 2411 gånger
Shirai: 2295 gånger
Hoban: 2175 gånger


In [5]:
theta.to_parquet("theta.parquet.gzip", compression="gzip")
alfa_x.to_parquet("alfax.parquet.gzip", compression="gzip")
beta_x.to_parquet("betax.parquet.gzip", compression="gzip")
alfa_y.to_parquet("alfay.parquet.gzip", compression="gzip")
beta_y.to_parquet("betay.parquet.gzip", compression="gzip")